In [1]:
# Import the packages and functions we will need
import numpy as np

from scipy.special import logsumexp

All material needed for this notebook can be found in the public GitHub repository https://github.com/dchoyle/datascience_notes

## Examples - Using scipy.special.logsumexp for simple log-sum-exp calculations

First of all let's demonstrate an extreme example. We'll use an array $a = [70.0, 2.7, 20.3, 15.9, -2.3]$. In this example, the value of $70.0$ will be so dominant when we exponentiate it that adding the exponential of any of the other values in the array will largely be irrelevant and the "log-sum-exp" function will return 70.0. Let's check.

In [2]:
# Create the array of the a_k values
a1 = np.array([70.0, 2.7, 20.3, 15.9, -2.3])

# Use the scipy.special log-sum-exp function
lse1 = logsumexp(a1)

# Look at the result
lse1

70.0

Yes, we got a result of 70. Hopefully that give you an idea of what is going on with the "log-sum-exp" trick logic, but let's try an example where at least one other value of $a_{k}$ makes a difference to the sum.

In [3]:
# Create the array of the a_k values
a2 = np.array([70.0, 68.9, 20.3, 72.9, 40.0])

# Use the scipy.special log-sum-exp function
lse2 = logsumexp(a2)

# Look at the result
lse2

72.9707742189605

We can see how the result we get is dominated by the entry of 72.9 in the starting array, but not precisely equivalent to it. The values of 70.0 and 68.9 have made a small contribution to the sum, so we get a final value of 72.97. On the other hand, the values of 20.3 and 40.0 made no contribution. We can confirm this by re-running the calculation without the values of 20.3 and 40.0 in the starting array. Let's find out.

In [4]:
# Create the array of the a_k values
a3 = np.array([70.0, 68.9, 72.9])

# Use the scipy.special log-sum-exp function
lse3 = logsumexp(a3)

# Look at the result
lse3

72.9707742189605

Yes, we get the same result as before, 72.9707742189605, when we explicitly exclude the values of 20.3 and 40.0 from our starting array.

## Examples - including signs in the summation

What I really like about the SciPy implementation of the logsumexp function is that you can also include the signs of the various contributions to the summation. Note this is the signs of the values of $\exp(a_{k})$, not the signs of the $a_{k}$. So let's say we wanted to compute $\exp(a_{1})\;-\; \exp(a_{2})\;+\;\exp(a_{3})$, we can easily do that by passing an array of signs, $b=[1.0, -1.0, 1.0]$ to the SciPy logsumexp function. We'll demonstrate that below. 

In [5]:
# Create the array of the a_k values
a4 = np.array([10.0, 9.99999, 1.2])
b4 = np.array([1.0, -1.0, 1.0])

# Use the scipy.special log-sum-exp function
lse4 = logsumexp(a=a4, b=b4)

# Look at the result
lse4

1.2642342014146895

If you look at the output of the example above you'll see that the final result is much closer to the value of the last array element $a_{3} = 1.2$. This is because the first two contributions, $\exp(a_{1})$ and $\exp(a_{2})$ almost cancel each other out because the contribution $\exp(a_{2})$ is pre-fixed by a factor of -1. What we get left with is something close to $\log(\exp(a_{3}))\;=\; a_{3}$. 

There is also a small subtlety in using the SciPy logsumexp function with signed contributions. If the substraction of some terms had led to an overall negative result, scipy.special.logsumexp will rerturn NaN as the result. In order to get it to always return a result for us, we have to tell it to return the sign of the final summation as well, by setting the return_sign argument of the function to True. We demonstrate this below.

In [6]:
# Create the array of the a_k values
a5 = np.array([10.0, 10.99999, 1.2])
b5 = np.array([1.0, -1.0, 1.0])

# Use the scipy.special log-sum-exp function
lse5 = logsumexp(a=a5, b=b5, return_sign=True)

# Look at the result
lse5

(10.54122130642536, -1.0)

You can see from the form of the output that we now get two values in the returned result. The first value is the (approximate) value of the magnitude of logsumexp, i.e.  $\log \left ( \left | \sum_{k} \exp(a_{k}) \right |\right )$, whilst the second value is the sign of the summation, i.e. ${\rm sign} \left ( \sum_{k}\exp(a_{k}) \right )$. Because in this example we've increased the value of $a_{2}$ to be larger than the value of $a_{1}$, we get a negative value overall, as indicated by the -1.0 in the second element of the output tuple.

If we'd called the logsumexp function without the argumemt return_sign=True, we would have got NaN as the output. Whenever I'm passing in an array of signs $b$ into the SciPy logsumexp function I always set return_sign=True even if I expect the final result to be positive. Doing so stops me from getting a NaN output in situations where my intuition about getting a positive result was wrong. I just have to remember that whenever I set return_sign=True I will get a tuple returned as the output, not a single float.